# LBM Step 3: Segmentation

## Segmentation: Extract neuronal locations and planar time-traces.

- Apply the constrained nonnegative matrix factorization (CNMF) source separation algorithm to extract initial estimates of neuronal spatial footprints and calcium traces.
- Apply quality control metrics to evaluate the initial estimates, and narrow down to the final set of estimates.

# Caiman docs on component eval

https://caiman.readthedocs.io/en/latest/Getting_Started.html#component-evaluation

> The quality of detected components is evaluated with three parameters:
>
> Spatial footprint consistency (rval): The spatial footprint of the component is compared with the frames where this component is active. Other component’s signals are subtracted from these frames, and the resulting raw data is correlated against the spatial component. This ensures that the raw data at the spatial footprint aligns with the extracted trace.
>
> Trace signal-noise-ratio (SNR): Peak SNR is calculated from strong calcium transients and the noise estimate.
>
> CNN-based classifier (cnn): The shape of components is evaluated by a 4-layered convolutional neural network trained on a manually annotated dataset. The CNN assigns a value of 0-1 to each component depending on its resemblance to a neuronal soma.
> 
> Each parameter has a low threshold:
> - (rval_lowest (default -1), SNR_lowest (default 0.5), cnn_lowest (default 0.1))
>
> and high threshold
> 
> - (rval_thr (default 0.8), min_SNR (default 2.5), min_cnn_thr (default 0.9))
> 
> A component has to exceed ALL low thresholds as well as ONE high threshold to be accepted.

In [2]:
import sys
from pathlib import Path
import os
import numpy as np
import zarr

import logging
import mesmerize_core as mc
import matplotlib.pyplot as plt

try:
    import cv2
    cv2.setNumThreads(0)
except():
    pass

logging.basicConfig()

from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

os.environ["CONDA_PREFIX_1"] = ""
if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

raw_data_path = Path().home() / "caiman_data"
movie_path = raw_data_path / 'animal_01' / "session_01" / 'save_gui.zarr'


2024-09-06 02:42:16.355980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-06 02:42:16.356008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 02:42:16.356358: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Paths (TODO)

```
/
    └── 
        ├── a
        │   ├── 
        │   │   ├── 
        │   │   ├── 
        │   │   └── 
        │   ├── 
        │   │   ├── 
        │   │   └── 
        │   └── 
        └── 
            ├──
            │   ├── 
            │   ├── 
            │   └── 
            ├── 
            └── 
```

**For this demo set the `caiman_data` dir as the parent raw data path**

### Batch path, this is where caiman outputs will be organized, same as the previous step

This can be anywhere, it does not need to be under the parent raw data path.

In [3]:
batch_path = raw_data_path / 'batch.pickle'
mc.set_parent_raw_data_path(str(raw_data_path))

# you could alos load the registration batch and 
# save this patch in a new dataframe (saved to disk automatically)
try:
    df = mc.load_batch(batch_path)
except (IsADirectoryError, FileNotFoundError):
    df = mc.create_batch(batch_path)

df=df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 9781250d-22cb-4ebc-9d...,2024-09-06T00:19:51,2024-09-06T00:33:05,764.98 sec,None,9781250d-22cb-4ebc-9de1-ad97e101ddee
1,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 5ffcad60-5506-4243-93...,2024-09-06T00:19:51,2024-09-06T00:35:31,142.87 sec,None,5ffcad60-5506-4243-936c-f8473ff4ab50
2,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 0a463c4f-985d-47a4-9e...,2024-09-06T00:19:51,2024-09-06T00:48:21,766.17 sec,None,0a463c4f-985d-47a4-9e95-cf776d98a1a3
3,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 8cb260f3-ced8-4433-b9...,2024-09-06T00:19:51,2024-09-06T00:50:47,142.94 sec,None,8cb260f3-ced8-4433-b9bf-fd6116b20964


# CNMF

Perform CNMF using the mcorr output.

Similar to mcorr, put the CNMF params within the `main` key. The `refit` key will perform a second iteration, as shown in the `caiman` `demo_pipeline.ipynb` notebook.

In [4]:
# some params for CNMF
params_cnmf = {
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate, very important!
            'p': 1,
            'nb': 2,
            'merge_thr': 0.85,
            'rf': 15,
            'stride': 6, # "stride" for cnmf, "strides" for mcorr
            'K': 4,
            'gSig': [4, 4],
            'ssub': 1,
            'tsub': 1,
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

# Add CNMF item

You can provide the mcorr item row to `input_movie_path` and it will resolve the path of the input movie from the entry in the DataFrame.

In [5]:
good_mcorr_index = 3

# add a batch item
df.caiman.add_item(
    algo='cnmf', # algo is cnmf
    input_movie_path=df.iloc[good_mcorr_index],  # use mcorr output from a completed batch item
    params=params_cnmf,
    item_name=df.iloc[good_mcorr_index]["item_name"], # use the same item name
)

See the cnmf item at the bottom of the dataframe

In [6]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 9781250d-22cb-4ebc-9d...,2024-09-06T00:19:51,2024-09-06T00:33:05,764.98 sec,None,9781250d-22cb-4ebc-9de1-ad97e101ddee
1,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 5ffcad60-5506-4243-93...,2024-09-06T00:19:51,2024-09-06T00:35:31,142.87 sec,None,5ffcad60-5506-4243-936c-f8473ff4ab50
2,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 0a463c4f-985d-47a4-9e...,2024-09-06T00:19:51,2024-09-06T00:48:21,766.17 sec,None,0a463c4f-985d-47a4-9e95-cf776d98a1a3
3,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 8cb260f3-ced8-4433-b9...,2024-09-06T00:19:51,2024-09-06T00:50:47,142.94 sec,None,8cb260f3-ced8-4433-b9bf-fd6116b20964
4,cnmf,save_gui,8cb260f3-ced8-4433-b9bf-fd6116b20964/8cb260f3-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",None,2024-09-06T02:43:35,None,None,None,2570aef8-1cca-4d71-af88-ce64cfbfaf65


# Run CNMF

The API is identical to running mcorr

In [7]:
index = -1  # most recently added item
df.iloc[index].caiman.run()

2024-09-06 02:43:47.605069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-06 02:43:47.605096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 02:43:47.605444: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                                    save_gui
input_movie_path    8cb260f3-ced8-4433-b9bf-fd6116b20964/8cb260f3-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
added_time                                        2024-09-06T02:43:35
ran_time                                                         None
algo_duration                                                    None
comments                                                         None
uuid                             2570aef8-1cca-4d71-af88-ce64cfbfaf65
Name: 4, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, '

The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman


making memmap


performing CNMF
fitting images
refitting


# Reload dataframe

In [ ]:
df = df.caiman.reload_from_disk()
df

# CNMF outputs

Similar to mcorr, you can use the `mesmerize-core` API to fetch the outputs. The API reference for CNMF is here: https://mesmerize-core.readthedocs.io/en/latest/api/cnmf.html

In [ ]:
index = -1  # index of the cnmf item, last item in the dataframe

# temporal components
temporal = df.iloc[index].cnmf.get_temporal()

In [ ]:
temporal.shape

Many of the cnmf functions take a rich set of arguments

In [ ]:
# get accepted or rejected components
temporal_good = df.iloc[index].cnmf.get_temporal("good")

# shape is [n_components, n_frames]
temporal_good.shape

In [ ]:
# get specific components
df.iloc[index].cnmf.get_temporal(np.array([1, 5, 9]))

In [ ]:
# get temporal with the residuals, i.e. C + YrA
temporal_with_residuals = df.iloc[index].cnmf.get_temporal(add_residuals=True)

In [ ]:
# get contours
contours = df.iloc[index].cnmf.get_contours()

Returns: `(list of np.ndarray of contour coordinates, list of center of mass)`

In [ ]:
print(f"contour 0 coordinates:\n\n{contours[0][0]}\n\n com: {contours[1][0]}")

In [ ]:
len(contours)

In [ ]:
# get_contours() also takes arguments
contours_good = df.iloc[index].cnmf.get_contours("good")

In [ ]:
len(contours_good[0]) # number of contours

swap_dim

In [ ]:
# get the first contour using swap_dim=True (default)
swap_dim_true = df.iloc[index].cnmf.get_contours()[0][0]

In [ ]:
# get the first contour  with swap_dim=False
swap_dim_false = df.iloc[index].cnmf.get_contours(swap_dim=False)[0][0]

In [ ]:
plt.plot(
    swap_dim_true[:, 0], 
    swap_dim_true[:, 1], 
    label="swap_dim=True"
)
plt.plot(
    swap_dim_false[:, 0], 
    swap_dim_false[:, 1], 
    label="swap_dim=False"
)
plt.legend()

In [ ]:
# swap_dim swaps the x and y dims
plt.plot(
    swap_dim_true[:, 0], 
    swap_dim_true[:, 1], 
    linewidth=30
)
plt.plot(
    swap_dim_false[:, 1], 
    swap_dim_false[:, 0], 
    linewidth=10
)

# Reconstructed movie - `A * C`
# Reconstructed background - `b * f`
# Residuals - `Y - AC - bf` 

Mesmerize-core provides these outputs as lazy arrays. This allows you to work with arrays that would otherwise be hundreds of gigabytes or terabytes in size.

In [ ]:
rcm = df.iloc[index].cnmf.get_rcm()
rcm

LazyArrays behave like numpy arrays

In [ ]:
rcm[42]

In [ ]:
rcm[10:20].shape

# Using LazyArrays

In [ ]:
rcm_accepted = df.iloc[index].cnmf.get_rcm("good")
rcm_rejected = df.iloc[index].cnmf.get_rcm("bad")

In [ ]:
iw_max = fpl.ImageWidget(
    data=[rcm_accepted.max_image, rcm_rejected.max_image],
    names=["accepted", "rejected"],
    grid_plot_kwargs={"size": (900, 450)},
    cmap="gnuplot2"
)
iw_max.show()

In [ ]:
iw_rcm_separated = fpl.ImageWidget(
    data=[rcm_accepted, rcm_rejected],
    names=["accepted", "rejected"],
    grid_plot_kwargs={"size": (900, 450)},
    cmap="gnuplot2"
)
iw_rcm_separated.show()

# All CNMF LazyArrays

In [ ]:
rcb = df.iloc[index].cnmf.get_rcb()
residuals = df.iloc[index].cnmf.get_residuals()
input_movie = df.iloc[index].caiman.get_input_movie()

`ImageWidget` accepts arrays that are sufficiently numpy-like

In [ ]:
iw_rcm = fpl.ImageWidget(
    data=[input_movie, rcm, rcb, residuals], 
    grid_plot_kwargs={"size": (800, 600)}, 
    cmap="gnuplot2"
)
iw_rcm.show()

In [ ]:
iw_rcm.close()

# Visualize everything with `mesmerize-viz`

In [ ]:
viz_cnmf = df.cnmf.viz()
viz_cnmf.show()

In [ ]:
viz_cnmf.close()

# Parameter Gridsearch

As shown for motion correction, the purpose of `mesmerize-core` is to perform parameter searches

In [ ]:
# itertools.product makes it easy to loop through parameter variants
# basically, it's easier to read that deeply nested for loops
from itertools import product

# variants of several parameters
gSig_variants = [4, 6]
K_variants = [4, 8]
merge_thr_variants = [0.8, 0.95]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmf)

# create a parameter grid
parameter_grid = product(gSig_variants, K_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for gSig, K, merge_thr in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    new_params_cnmf["main"]["gSig"] = [gSig, gSig]
    new_params_cnmf["main"]["K"] = K
    new_params_cnmf["main"]["merge_thr"] = merge_thr
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmf",
        item_name=df.iloc[good_mcorr_index]["item_name"],  # good mcorr item
        input_movie_path=df.iloc[good_mcorr_index],
        params=new_params_cnmf
    )

We now have lot of cnmf items

In [ ]:
df

View the diffs

In [ ]:
df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[-1]["item_name"])

# Run the `cnmf` batch items

In [ ]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip
        
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

# Load outputs

In [ ]:
df = df.caiman.reload_from_disk()

In [ ]:
df

# Visualize with `mesmerize-viz`

In [ ]:
viz_cnmf = df.cnmf.viz()
viz_cnmf.show(sidecar=True)

# Caiman docs on component eval

https://caiman.readthedocs.io/en/latest/Getting_Started.html#component-evaluation

> The quality of detected components is evaluated with three parameters:
>
> Spatial footprint consistency (rval): The spatial footprint of the component is compared with the frames where this component is active. Other component’s signals are subtracted from these frames, and the resulting raw data is correlated against the spatial component. This ensures that the raw data at the spatial footprint aligns with the extracted trace.
>
> Trace signal-noise-ratio (SNR): Peak SNR is calculated from strong calcium transients and the noise estimate.
>
> CNN-based classifier (cnn): The shape of components is evaluated by a 4-layered convolutional neural network trained on a manually annotated dataset. The CNN assigns a value of 0-1 to each component depending on its resemblance to a neuronal soma.
> 
> Each parameter has a low threshold:
> - (rval_lowest (default -1), SNR_lowest (default 0.5), cnn_lowest (default 0.1))
>
> and high threshold
> 
> - (rval_thr (default 0.8), min_SNR (default 2.5), min_cnn_thr (default 0.9))
> 
> A component has to exceed ALL low thresholds as well as ONE high threshold to be accepted.

# This rich visualization is still customizable!

Public attributes:

- `image_widget`: the `ImageWidget` in the visualization
- `plot_temporal`: the `Plot` with the temporal
- `plot_heatmap`: the `Plot` with the heatmap
- `cnmf_obj`: The cnmf object currently being visualized. This object gets saved to disk when you click the "Save Eval to disk" button.
- `component_index`: current component index, `int`

A few public methods:
- `show()` show the visualization
- `set_component_index(index: int)` manually set the component index

In [ ]:
viz_cnmf.image_widget.cmap = "gray"

In [ ]:
viz_cnmf.plot_heatmap

In [ ]:
viz_cnmf.plot_heatmap["heatmap"].cmap = "viridis"

In [ ]:
viz_cnmf.plot_heatmap["heatmap"].cmap.vmax

In [ ]:
viz_cnmf.plot_heatmap["heatmap"].cmap.vmax = 2_000

Customize contours

In [ ]:
for subplot in viz_cnmf.image_widget.gridplot:
    subplot["contours"][:].thickness = 1.0

In [ ]:
for subplot in viz_cnmf.image_widget.gridplot:
    subplot["contours"].visible = False

In [ ]:
for subplot in viz_cnmf.image_widget.gridplot:
    subplot["contours"].visible = True

In [ ]:
viz_cnmf.plot_temporal["line"].thickness()

In [ ]:
viz_cnmf.plot_temporal["line"].thickness = 1

# Visualize fewer things

In [ ]:
viz_simple = df.cnmf.viz(
    image_data_options=["input", "rcm"],
)
viz_simple.show(sidecar=True)

# More customization of kwargs

In [ ]:
viz_more_custom = df.cnmf.viz(
    image_data_options=["input", "rcm", "rcm-max", "corr"],
    temporal_kwargs={"add_residuals": True},
)

In [ ]:
viz_more_custom.show(sidecar=True)